TransMed Translator is using Ginas data as a test bedsince none of the groups has ingested it yet.

see:  
https://github.com/NCATS-Tangerine/ncats-ingest/issues/1

The best source for the data to date is found at:  
https://tripod.nih.gov/ginas/#/gsrs/data

(commented out after first fetch)

In [2]:
# wget -q --timestamping https://tripod.nih.gov/ginas/gsrs/fullSeedData-2016-06-16.gsrs

In [4]:
ls -lh

total 91M
-rw-r--r-- 1 tomc tomc  407 Dec 12 14:14 200161212_ingest_call.txt
-rw-r--r-- 1 tomc tomc  338 Jan  4 13:04 20170104_recap_call.txt
-rw-r--r-- 1 tomc tomc  91M Aug 12 12:37 fullSeedData-2016-06-16.gsrs
-rw-r--r-- 1 tomc tomc 1.5K Jan 11 19:36 Untitled.ipynb


In [6]:
file fullSeedData-2016-06-16.gsrs

fullSeedData-2016-06-16.gsrs: gzip compressed data, last modified: Thu Jun 16 14:53:25 2016, from Unix


In [13]:
gunzip - < fullSeedData-2016-06-16.gsrs > fullSeedData-2016-06-16

In [18]:
file fullSeedData-2016-06-16

fullSeedData-2016-06-16: ASCII text, with very long lines


make sure it is not all on one half gigabyte line before taking alook (happens).

In [12]:
wc -l < fullSeedData-2016-06-16

84979


In [19]:
head -2 fullSeedData-2016-06-16

WORKED: 18MXK3D6DB	b029367a-c360-4617-8a2e-fd90c7a88fdf	{"structure":{"references":["17162b41-9aa8-4549-b423-a1169d864351","383105ca-e739-4df4-9be3-cb7342556d73"],"molfile":"\n  Symyx   06151614132D 1   1.00000     0.00000     0\n\n 12 12  0     0  0            999 V2000\n   21.4377   -8.1424    0.0000 C   0  0  0  0  0  0           0  0  0\n   22.4606   -8.7330    0.0000 C   0  0  0  0  0  0           0  0  0\n   23.4835   -8.1424    0.0000 C   0  0  0  0  0  0           0  0  0\n   23.4835   -6.9613    0.0000 C   0  0  0  0  0  0           0  0  0\n   22.4606   -9.9141    0.0000 C   0  0  0  0  0  0           0  0  0\n   21.4377  -10.5046    0.0000 C   0  0  0  0  0  0           0  0  0\n   21.4377  -11.6857    0.0000 S   0  0  3  0  0  0           0  0  0\n   22.6188  -11.6857    0.0000 O   0  0  0  0  0  0           0  0  0\n   20.2566  -11.6857    0.0000 O   0  0  0  0  0  0           0  0  0\n   21.4377  -12.8668    0.0000 O   0  0  0  0  0  0           0  0  0\n   20.4149   -9.9

looks like an array of:   

    WORKED: <SRS_UNII>	<UUID> {jason: blob}
    

 __ASIDE__  
Where SRS_ID is FDA's "Substance Registration System Unique Ingrediant Identifier"  
The second one for example, goes to:   
    https://fdasis.nlm.nih.gov/srs/auto/startswith/o061w4c37h

(Will also note that although Ginas does not have an "InChIKey" for this substance the fda does associate the inchikey "DHAZIUXMHRHVMP-UHFFFAOYSA-N"

FDA has https://fdasis.nlm.nih.gov/srs/download/srs/UNIIs.zip

with 601k rows of:  

    head -1 "UNIIs 10Nov2016 Names.txt"
    NAME	TYPE	UNII	PT
    
and https://fdasis.nlm.nih.gov/srs/download/srs/UNII_Data.zip

with 83k rows of:  

    head -1 "UNIIs 10Nov2016 Records.txt"
    UNII	PT	RN	EC	NCIT	RXCUI	ITIS	NCBI	PLANTS	GRIN	INN_ID	MF	INCHIKEY	SMILES	UNII_TYPE

along with 8k rows in "UNIIs 10Nov2016 Changes.txt" with:  
    
    UNII	DATA	FIELD	TYPE
    

Seems the UNIIs Records file could be a resource for filling in gaps.  
There are 48,341 unii->inchikey mappings and 35,385 unii missing mappings.

Although this file contains json it is not quite valid in its current form.

In [20]:
jq . fullSeedData-2016-06-16

parse error: Invalid numeric literal at line 1, column 7


: 4

Omitting the leading items or changing them to json are the options. Omitting is risky because although the few I have looked at the UNII and UUID are replicated in the json I can't be sure it is consistant so including them is safer.  

In [26]:
awk 'BEGIN{print "["}{$1="\"unid\": ";$2="\"" $2"\",";$3="\"ginas\": \""$3"\",";$4="\"record\": "$4;print "{" $0 "},"}END{print "]"}' fullSeedData-2016-06-16 > fullSeedData-2016-06-16.json


In [27]:
jq . fullSeedData-2016-06-16.json

parse error: Invalid numeric literal at line 33442, column 96


: 4

That is anoying. Malformed json deep in the file.

In [28]:
sed -n '33442'p fullSeedData-2016-06-16.json | cut -c1-200

{"unid":  "[ingredient]", "ginas": "3786568514", "record": 90061b24-2d13-4a1b-bfbe-46b22cf2cea2 {"substanceClass":"concept","names":[{"name":"HYDROCODONE POLISTIREX","stdName":"HYDROCODONE POLISTIREX"


Everything is displaced to the right by one with the appearence of `[ingredient]`.

In [40]:
grep -c "^WORKED: \[ingredient\]" fullSeedData-2016-06-16

4933


In [41]:
grep -c "^WORKED: \[" fullSeedData-2016-06-16

4933


Appears the 5k with `[ingredient]` are the only ones with the optional flag in that location.  


In [29]:
awk 'BEGIN{print "["}\
NR==1             {$1="\"unid\":";$2="\"" $2 "\",";$3="\"ginas\": \"" $3 "\",";$4="\"record\":" $4;print "{"$0"}"}\
NR>1&&\
$2!="[ingredient]"{$1="\"unid\":";$2="\"" $2 "\",";$3="\"ginas\": \"" $3 "\",";$4="\"record\":" $4;print ",{"$0"}"}\
$2=="[ingredient]"{$1="\"unid\":";$3="\"" $3 "\",";$4="\"ginas\": \"" $4 "\",";$5="\"ingredient\":\"true\",\"record\":" $5;$2="";print ",{"$0"}"}\
END{print "]"}' fullSeedData-2016-06-16 > fullSeedData-2016-06-16.json

In [23]:
jq . fullSeedData-2016-06-16.json > fullSeedData-2016-06-16_pp.json

In [38]:
head fullSeedData-2016-06-16_pp.json

[
  {
    "unid": "18MXK3D6DB",
    "ginas": "b029367a-c360-4617-8a2e-fd90c7a88fdf",
    "record": {
      "structure": {
        "references": [
          "17162b41-9aa8-4549-b423-a1169d864351",
          "383105ca-e739-4df4-9be3-cb7342556d73"
        ],


How are records spaced out after formatting?

In [36]:
grep -n "^  {" fullSeedData-2016-06-16_pp.json| head

2:  {
146:  {
355:  {
603:  {
822:  {
1200:  {
1626:  {
1905:  {
2199:  {
2467:  {
grep: write error: Broken pipe


In [ ]:
Looks like a couple of hundred lines each.
What does a record look like?

In [4]:
sed -n '355,603'p fullSeedData-2016-06-16_pp.json

  {
    "unid": "6V3I57K9UL",
    "ginas": "1c626f08-39eb-42c1-86b2-6897b5009106",
    "record": {
      "structure": {
        "references": [
          "31f8a7c9-9e0c-41c4-b7ba-66df828e9a5c",
          "6dd38620-f956-4c2f-b17b-ec6cc78bfcb5"
        ],
        "molfile": "\n Symyx 06151614132D 1 1.00000 0.00000 0\n\n 22 22 0 0 0 999 V2000\n 11.2542 -3.4792 0.0000 C 0 0 0 0 0 0 0 0 0\n -0.3208 -3.4792 0.0000 C 0 0 0 0 0 0 0 0 0\n 11.2375 -2.1292 0.0000 O 0 0 0 0 0 0 0 0 0\n -1.4708 -4.1250 0.0000 C 0 0 0 0 0 0 0 0 0\n -0.3208 -2.1459 0.0000 C 0 0 0 0 0 0 0 0 0\n -2.6542 -2.1292 0.0000 C 0 0 0 0 0 0 0 0 0\n 0.8292 -4.1459 0.0000 C 0 0 3 0 0 0 0 0 0\n -2.6542 -3.4959 0.0000 C 0 0 0 0 0 0 0 0 0\n -1.4833 -1.4834 0.0000 C 0 0 0 0 0 0 0 0 0\n 12.4042 -4.1250 0.0000 O 0 0 0 0 0 0 0 0 0\n -3.7958 -1.4834 0.0000 I 0 0 0 0 0 0 0 0 0\n 10.0875 -4.1084 0.0000 C 0 0 0 0 0 0 0 0 0\n 1.9917 -3.4792 0.0000 C 0 0 0 0 0 0 0 0 0\n 13.5667 -3.4792 0.0000 C 0 0 0 0 0 0 0 0 0\n 0.8500 -5.4584 0.0000 C 0 0 

parsed into ntriples with python. see: (./Parse_Ginas_json.py)[Parse_Ginas_json.py]  jupyter notebook.

In [27]:
wc -l < ginas.nt

3522185


RDF ntriples have lines of the form "`<subject> <predicate> <object> .`"  
we should be able to confirm certain conditions hold:  
\>    The subject is always a Ginias identifier.  
\>    The predicate is now always a path to root (to be reinterperted as an ontological term).  
\>    The object is most variable but when it is a Ginas identifier. That identifier exists as a subject.  

In [34]:
grep -c "^<GINAS:" ginas.nt

3522185


In [33]:
cut -f2 -d' ' ginas.nt| sort | uniq -c | sort -nr

 649506 <reference_uuid:>
 649506 <reference_docType:>
 649506 <reference_citation:>
 553496 <reference_tag:>
 205757 <<name_references:>
 198471 <codes_code:>
 113086 <structure_references:>
 106366 <reference_url:>
  84979 <unii:>
  84979 <substanceClass:>
  84977 <names_stdName:>
  70718 <code_references:>
  56543 <structure_formula:>
   7437 <mixture_component_substance:>
   4933 <ingredient:>
   1925 <mixture_component_type:>


In [32]:
cut -f1 -d' ' ginas_s.nt| sort -u > ginas_subject.list

In [20]:
wc -l < ginas_subject.list

734485


In [22]:
cut -f3 -d' ' ginas_s.nt |grep "^<GINAS:" | sort -u > ginas_object.list

In [24]:
wc -l < ginas_object.list

656364


In [26]:
comm -13 ginas_subject.list ginas_object.list

No output means all objects exist as subjects which is good.  

It may be I should split the GINAS: curie into substance and reference  

